In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json

In [2]:
#CodeRead

data_folder = '../Data_contest/dataset/'


genome_scores_df=pd.read_csv(data_folder+'genome_scores.csv') # Large (500MB)
movies_df=pd.read_csv(data_folder+'movies.csv')
train_df=pd.read_csv(data_folder+'train.csv') # Large 500MB
validation_df = pd.read_csv(data_folder+'validation.csv') 








In [3]:
#CodeRead

# create movie rating dataset from train

# Feature vector for the 10000 movies, each with a 1128 dimensional vector. 
# If a movie doesn't appear in genome_scores we make it simply the 0 vector.
X=np.zeros((10000,1128)) 
movies_with_featvecs=set(genome_scores_df['movieId'])
# The average rating, for each of the movies in the training set. 
# -1 if it is not in the train set.
rating_movies = -1*np.ones(10000) 

# Each movie, is labelled +1 or -1 based on whetherr it is a comedy or not
comedy = np.zeros(10000) 

for i in range(10000):
    if i not in movies_with_featvecs:
        continue
    temp = genome_scores_df[genome_scores_df['movieId']==i]
    feat_vec= np.array(temp['relevance'])
    X[i,:]=feat_vec

for i in range(10000):
    temp = train_df[train_df['movieId']==i]
    if len(temp)==0:
        continue
    ratings_curr_movies = temp['rating']
    rating_movies[i] = np.mean(ratings_curr_movies)

for i in range(10000):
    temp = movies_df[movies_df['movieId']==i]
    if len(temp)==0:
        continue
    temp = temp['genres'].values[0]
    temp = temp.split('|')
    comedy[i] = 'Comedy' in temp
    comedy[i] = 2*comedy[i] -1


In [4]:
np.array(train_df['rating']).mean()

3.3601078084681526

# Problem 1. K-Nearest Neighbours

Run k-Nearest neighbours on the binary classification dataset below, for classifiying whether a given movie is a comedy or not. The train and test feature vectors are given by X_train, X_test and the train and test ratings are Y_train and Y_test. 

Run k-Nearest neighbours for different k values (1,2,4,8,16,32,64) and report accuracies in the next cell. 

In [42]:
#CodeWrite cell

X_train = np.array(X[:8000,])
X_test = np.array(X[8000:])

Y_train=np.array(comedy[:8000])
Y_test=np.array(comedy[8000:])
Y_test_pred = np.empty((Y_test.shape))

sorted_idx = np.empty((X_test.shape[0],X_train.shape[0]))
for i in range(X_test.shape[0]):
    test_instance = X_test[i]
    dist = np.sqrt(np.sum((X_train - test_instance)**2, axis=1))
    sorted_idx[i] = np.argsort(dist)

    
ks = [1,2,4,8,16,32,64]

for k in ks:
    for i in range(X_test.shape[0]):
        neighbor_idx = sorted_idx[i,:k].astype(int)
        neighbor_ratings = Y_train[neighbor_idx]
        Y_test_pred[i] = np.sign(np.mean(neighbor_ratings))

    corr = np.where(Y_test_pred == Y_test, 1,0)
    acc = np.mean(corr)
#     print('k ',k,', test accuracy ', acc)


TextWrite cell. Report test accuracies for different k here.

### Test Accuracies for different k values

|      k     | Test Accuracy | 
|------------|---------------|
| 1 | 0.841 |
| 2 | 0.7475 |
| 4 | 0.8085 |
| 8 | 0.834 |
| 16 | 0.8025 |
| 32 | 0.855 |
| 64 | 0.854 |

